# Two-View 3D Reconstruction

This notebook implements a simple two-view Structure from Motion approach to create a 3D reconstruction from two images (01.jpg and 02.jpg).

## Setup and Requirements

In [ ]:
import sys
import os
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  if os.path.exists('/content/Prague_ml_data'):
    print("Prague_ml_data already exists")
  else:
    ! git clone https://github.com/VarunBurde/Prague_ml_data

In [ ]:
# Install required packages
!pip install open3d networkx matplotlib tqdm opencv-python

In [ ]:
import cv2
import open3d as o3d
import os
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# For displaying Open3D visualizations in notebooks
from google.colab.patches import cv2_imshow
import base64
from IPython.display import HTML

def display_open3d_to_notebook(vis, width=900, height=600):
    vis.update_renderer()
    image = vis.capture_screen_float_buffer()
    image_array = np.asarray(image)
    image_array = (image_array * 255).astype(np.uint8)
    image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    _, encoded_img = cv2.imencode('.png', image_array)
    encoded_img = base64.b64encode(encoded_img)
    html = f'<img src="data:image/png;base64,{encoded_img.decode()}" width="{width}" height="{height}"/>'
    return HTML(html)

## 1. Data Loading

First, let's load our two specific images: 01.jpg and 02.jpg.

In [ ]:
# For Colab: Upload images
from google.colab import files
import shutil

# Set the paths for our specific images
if IN_COLAB:
  img_path = "/content/Prague_ml_data/dataset/two_view_data/images"
  image_paths = [os.path.join(img_path, "01.jpg"), os.path.join(img_path, "02.jpg")]
  K = np.loadtxt("/content/Prague_ml_data/dataset/two_view_data/K.txt")
else:
  img_path = "dataset/two_view_data/images"
  image_paths = [os.path.join(img_path, "01.jpg"), os.path.join(img_path, "02.jpg")]
  K = np.loadtxt("dataset/two_view_data/K.txt")

plt.figure(figsize=(15, 7))
for i, path in enumerate(image_paths):
    plt.subplot(1, 2, i+1)
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(f"Image {i+1}: {os.path.basename(path)}")
plt.tight_layout()
plt.show()

## 2. Load Images and Extract Features

Now we'll load our two images and extract feature points using SIFT.

In [ ]:
# Load our two images
images = []
images_rgb = []

for path in image_paths:
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    images_rgb.append(img_rgb)

print(f"Loaded {len(images)} images")

# Create SIFT detector with more aggressive parameters for denser features
sift = cv2.SIFT_create(
    nfeatures=0,  # No limit on number of features
    contrastThreshold=0.04,  # Lower threshold for more features
    edgeThreshold=10,  # Higher threshold for more features
    sigma=1.6  # Default sigma
)

In [ ]:
# Detect features in both images
all_kp = []
all_des = []

for i, img_rgb in enumerate(images_rgb):
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_eq = cv2.equalizeHist(img_gray)
    
    # Detect features at multiple scales
    kp_standard, des_standard = sift.detectAndCompute(img_rgb, None)
    kp_eq, des_eq = sift.detectAndCompute(img_eq, None)
    
    # Combine features
    if des_standard is not None and des_eq is not None:
        kp = kp_standard + kp_eq
        des = np.vstack((des_standard, des_eq))
    elif des_standard is not None:
        kp = kp_standard
        des = des_standard
    elif des_eq is not None:
        kp = kp_eq
        des = des_eq
    else:
        kp = []
        des = np.array([])
    
    all_kp.append(kp)
    all_des.append(des)
    print(f"Image {i+1} ({os.path.basename(image_paths[i])}): Detected {len(kp)} keypoints")

### Visualize Detected Features

Let's visualize the keypoints detected in both images.

In [ ]:
# Display keypoints on both images
plt.figure(figsize=(15, 7))
for i in range(2):
    plt.subplot(1, 2, i+1)
    img_with_kp = cv2.drawKeypoints(images_rgb[i], all_kp[i], None, 
                                    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(img_with_kp)
    plt.title(f"Image {i+1} ({os.path.basename(image_paths[i])}): {len(all_kp[i])} keypoints")
plt.tight_layout()
plt.show()

## 3. Match Features Between Images

Now we'll match features between our two images to find correspondences.

In [ ]:
# Improved FLANN matcher setup
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=100)  # More checks for better accuracy
flann = cv2.FlannBasedMatcher(index_params, search_params)

# Function to perform feature matching with cross-check
def match_features(des1, des2, lowes_ratio=0.75, cross_check=True):
    # Forward matching (img1 -> img2)
    if des1 is None or des2 is None or len(des1) == 0 or len(des2) == 0:
        return []
    
    matches12 = flann.knnMatch(des1, des2, k=2)
    good_matches12 = []
    for m, n in matches12:
        if m.distance < lowes_ratio * n.distance:
            good_matches12.append(m)
    
    if not cross_check:
        return good_matches12
    
    # Backward matching (img2 -> img1) for cross-check
    matches21 = flann.knnMatch(des2, des1, k=2)
    good_matches21 = []
    for m, n in matches21:
        if m.distance < lowes_ratio * n.distance:
            good_matches21.append(m)
    
    # Cross-check: keep matches that are consistent in both directions
    cross_matches = []
    for match12 in good_matches12:
        for match21 in good_matches21:
            # Check if match is consistent in both directions
            if match12.queryIdx == match21.trainIdx and match12.trainIdx == match21.queryIdx:
                cross_matches.append(match12)
                break
    
    return cross_matches

# Get matched points from keypoints and matches
def get_matched_points(kp1, kp2, matches):
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    return pts1, pts2

In [ ]:
# Match features between our two images (01.jpg and 02.jpg)
idx1, idx2 = 0, 1  # Using image indices 0 and 1 (01.jpg and 02.jpg)
good_matches = match_features(all_des[idx1], all_des[idx2], lowes_ratio=0.75)


In [ ]:
print(f"Found {len(good_matches)} good matches between {os.path.basename(image_paths[0])} and {os.path.basename(image_paths[1])}")

# # Display matches for verification
# match_img = cv2.drawMatches(
#     images_rgb[idx1], all_kp[idx1], 
#     images_rgb[idx2], all_kp[idx2], 
#     good_matches[:100], None, 
#     flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
# )

#or

mg1 = images_rgb[idx1]
img2 = images_rgb[idx2]
h1, w1 = img1.shape[:2]
h2, w2 = img2.shape[:2]
matched_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
matched_img[:h1, :w1] = img1
matched_img[:h2, w1:w1 + w2] = img2

# Draw keypoints
for m in good_matches[:10]:
    pt1 = tuple(map(int, all_kp[idx1][m.queryIdx].pt))
    pt2 = tuple(map(int, all_kp[idx2][m.trainIdx].pt))
    pt2 = (int(pt2[0] + w1), int(pt2[1]))  # shift pt2 x-coord

    # Draw circles
    cv2.circle(matched_img, pt1, 20, (0, 255, 0), -1)  # radius 6, green
    cv2.circle(matched_img, pt2, 20, (0, 255, 0), -1)

    # Draw connecting line
    cv2.line(matched_img, pt1, pt2, (255, 0, 0),10)  # blue line, thickness 2

plt.figure(figsize=(15, 10))
plt.imshow(match_img)
plt.title(f'Matches between {os.path.basename(image_paths[0])} and {os.path.basename(image_paths[1])}')
plt.show()

## 4. Two-View Reconstruction

Now we'll use the matched features to perform 3D reconstruction.

In [ ]:
# Initialize camera poses (R|t) for each image
# First camera (01.jpg) is at origin with identity rotation
camera_poses = [np.hstack((np.eye(3), np.zeros((3, 1))))]
camera_matrices = [K @ camera_poses[0]]

# Structure to store 3D points and their 2D observations
point_cloud = []
point_colors = []

pts1, pts2 = get_matched_points(all_kp[idx1], all_kp[idx2], good_matches)

# Calculate essential matrix with robust parameters
E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=2.0)

# Recover pose for second camera (02.jpg)
_, R, t, mask = cv2.recoverPose(E, pts1, pts2, K, mask=mask)

# Set second camera pose
camera_poses.append(np.hstack((R, t)))
camera_matrices.append(K @ camera_poses[1])

print(f"Camera 1 (01.jpg) pose:\n{camera_poses[0]}")
print(f"\nCamera 2 (02.jpg) pose:\n{camera_poses[1]}")

# Filter points using mask
mask = mask.ravel() == 1
pts1_good = pts1[mask]
pts2_good = pts2[mask]

print(f"\nUsing {np.sum(mask)} inlier matches for triangulation")

In [ ]:
# Triangulate points
pts1_good = pts1_good.reshape(-1, 2).T
pts2_good = pts2_good.reshape(-1, 2).T
points_4D = cv2.triangulatePoints(camera_matrices[0], camera_matrices[1], pts1_good, pts2_good)
points_3D = points_4D / points_4D[3]  # Convert to Cartesian
points_3D = points_3D[:3, :].T

# Filter points by depth
valid_points = []
valid_colors = []

for i in range(points_3D.shape[0]):
    point = points_3D[i]
    # Check if point is in front of both cameras
    if point[2] > 0:
        # Get corresponding 2D points
        x1, y1 = pts1_good[:, i]
        
        # Extract color from first image
        x1_int, y1_int = int(round(x1)), int(round(y1))
        if 0 <= x1_int < images_rgb[idx1].shape[1] and 0 <= y1_int < images_rgb[idx1].shape[0]:
            color = images_rgb[idx1][y1_int, x1_int] / 255.0
            valid_points.append(point)
            valid_colors.append(color)

point_cloud = np.array(valid_points)
point_colors = np.array(valid_colors)

print(f"Reconstructed point cloud has {len(point_cloud)} points")

### Visualize Point Cloud
Let's visualize the 3D point cloud reconstructed from our two images.

In [ ]:
# Create Open3D point cloud for visualization
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Remove outliers for cleaner visualization
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

In [ ]:
def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()

        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((1.0, 0.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
            triangles = np.asarray(geometry.triangles)
            vertices = np.asarray(geometry.vertices)
            colors = None
            if geometry.has_triangle_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
                colors = tuple(map(tuple, colors))
            else:
                colors = (1.0, 0.0, 0.0)

            mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
            graph_objects.append(mesh_3d)

    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

In [ ]:
o3d.visualization.draw_geometries = draw_geometries # replace function
o3d.visualization.draw_geometries([pcd])